# plots

> Plot generators

In [ ]:
#| default_exp plots

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|exporti

from typing import Dict, List, Optional
from sklearn.metrics import roc_curve
from lir import Xy_to_Xn

import numpy as np 
from matplotlib import pyplot as plt 


In [ ]:
#| export

def plot_lr_distributions(results:Dict, experiment_directory, save_plots:bool = True, show: Optional[bool] = False):
    """
    Plots the 10log LRs generated for the two hypotheses by the fitted system.
    """
    predicted_log_lrs = np.log10(results["lrs_predicted"])
    plt.figure(figsize=(10, 10), dpi=100)
    points0, points1 = Xy_to_Xn(predicted_log_lrs, np.array(results['y_test']))
    plt.hist(points0, bins=20, alpha=.25, density=True)
    plt.hist(points1, bins=20, alpha=.25, density=True)
    plt.xlabel(r'$log_{10}$ LR')
    if save_plots:
        savefig = os.path.join(experiment_directory, "lr_distributions")
        plt.savefig(savefig, dpi=600)
        plt.close()
    if show:
        plt.show()

In [ ]:
#| export

def plot_ROC_curve(results:Dict, experiment_directory, save_plots:bool = True, show: Optional[bool] = False):

    norm_distances = np.asarray(results["test_norm_distances"])
    fpr, tpr, thresholds = roc_curve(results['y_test'], 1 - norm_distances)
    plt.figure(figsize=(10, 10), dpi=100)
    plt.plot(fpr, fpr, linestyle='--', label='No Skill')
    plt.plot(fpr, tpr, color='r', label=r'ROC curve')
    plt.xlabel('False positive rate (1 - specificity)')
    plt.ylabel('True positive rate (sensitivity)')
    plt.title('ROC curve')
    plt.legend()
    if save_plots:
        savefig = os.path.join(experiment_directory, "ROC_curve")
        plt.savefig(savefig, dpi=600)
        plt.close()
    if show:
        plt.show()

In [ ]:
#| export

def plot_tippett(results:Dict, experiment_directory, save_plots:bool = True, show: Optional[bool] = False):
        
    """
    Plots the 10log LRs in a Tippett plot.
    """

    predicted_log_lrs = np.log10(results["lrs_predicted"])

    xplot = np.linspace(
        start=np.min(predicted_log_lrs),
        stop=np.max(predicted_log_lrs),
        num=100
    )

    lr_0, lr_1 = Xy_to_Xn(predicted_log_lrs, np.array(results["y_test"]))

    perc0 = (sum(i > xplot for i in lr_0) / len(lr_0)) * 100
    perc1 = (sum(i > xplot for i in lr_1) / len(lr_1)) * 100

    plt.figure(figsize=(10, 10), dpi=600)
    plt.plot(xplot, perc1, color='b', label=r'LRs given $\mathregular{H_1}$')
    plt.plot(xplot, perc0, color='r', label=r'LRs given $\mathregular{H_2}$')
    plt.axvline(x=0, color='k', linestyle='--')
    plt.xlabel('Log likelihood ratio')
    plt.ylabel('Cumulative proportion')
    plt.title('Tippett plot')
    plt.legend()

    if save_plots:
        savefig = os.path.join(experiment_directory, "tippet_plot")
        plt.savefig(savefig, dpi=600)
        plt.close()
    if show:
        plt.show()

In [ ]:
#| export 

def plot_cllr(results:Dict, experiment_directory, enfsi_years: List[int], cllr_expert_per_year:Dict, embeddingModel, save_plots:bool = True, show: Optional[bool] = False):
    
    """
    Plots cllr value for ENFSI tests. It computes both cllr of automated systems with the cllrs from experts.
    If there is no ENFSI data, this graph does not show.

    # todo: save table with cllr results.
    """

    cllr_auto_df = pd.DataFrame(columns=['Year', 'Expert', 'Cllr'])
    cllr_exp_df = pd.DataFrame(columns=['Year', 'Expert', 'Cllr'])
    years = enfsi_years

    for year in years:
        for cllr_exp in cllr_expert_per_year[year]:
            cllr_exp_df = cllr_exp_df.append({'Year': str(year), 'LR Estimator': "Participant", 'Cllr': cllr_exp},
                                                ignore_index=True)

        cllr_auto = cllr_auto_per_year[year]
        cllr_auto_df = cllr_auto_df.append(
            {'Year': str(year), 'LR Estimator': embeddingModel, 'Cllr': cllr_auto},
            ignore_index=True)

    cllr_df = cllr_exp_df.append(cllr_auto_df)
    sns.set_style("whitegrid")
    sc_plot = sns.catplot(data=cllr_df, x="Year", y="Cllr", hue="LR Estimator",
                            palette=sns.color_palette(['orange', 'blue']))

    # sc_plot.set_title("Cllrs for Automated system and ENFSI participants")
    # sc_plot.set(xticks=[map(str, years)])

    if save_plots:
        savefig = os.path.join(experiment_directory, "cllr_experts")
        plt.savefig(savefig, dpi=600)
        plt.close()
    if show:
        plt.show()

In [ ]:
#|export

def plot_ece(results:Dict, experiment_directory, save_plots:bool = True) -> object:
        savefig = os.path.join(experiment_directory, "ECE_plot")
        plot(np.asarray(results["lrs_predicted"]), np.asarray(results["y_test"]), path=savefig, kw_figure={'figsize': (10, 10), 'dpi': 600})

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()